# FACE MASK DETECTOR

## Block of Code Used for Scraping Images 

In [ ]:
# Used for scraping the pictures for our datasets ( for training our Deep Learning Model ) from BingImageCrawler
'''
from icrawler.builtin import BingImageCrawler
for keyword in ['person face smiling']:
    crawler =  BingImageCrawler(
        parser_threads=2,
        downloader_threads=4,
        storage={'root_dir': 'C:/Users/Tarun/Desktop/comp/projects/face_mask_detector/dataset/training_set/no_mask_1'} #storage={root_dir':'C:/Users/Tarun/Desktop/comp/image1/{}'.format(keyword)}
    )
    crawler.crawl(keyword=keyword, max_num=1, min_size=(200, 200))  '''

## Training the Model

In [ ]:
from keras.models import Sequential #initialise the neural network as its a sequence of layers
from keras.layers import Conv2D #import convolutional layers
from keras.layers import MaxPooling2D #import pooling layers which have max function
from keras.layers import Flatten #import flatten layer
from keras.layers import Dense #add the layers to the ann

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu')) #32 is number of feature detectors having dimensions (3,3),input_shape is format into which images will be converted .. 3 channels (rgb) of 64 * 64 coloured pixels, relu function for non-colinearity

# Step 2 - Pooling .. reducing size of the feature maps
classifier.add(MaxPooling2D(pool_size = (2, 2))) #2,2 is used so that we dont loose information and also be precise in where features are detected

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu')) #no. of nodes = 128 ( in power of 2 ), number around 100 is good 
classifier.add(Dense(units = 1, activation = 'sigmoid')) # binary output

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # binary cross entropy algorithm used to distinguish between the 2 classes

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

from keras.preprocessing.image import ImageDataGenerator #preprocesses images to prevent overfitting and enrichs our training set without increasing the number of images

#used for getting different variations of our data (in this case pictures) as we only have a limited dataset

#ImageDataGenerator for training data
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

#ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Part 2 - Fitting the CNN to the images
classifier.fit_generator(training_set,
                         steps_per_epoch = 1000,
                         epochs = 1, #reduce number of epochs because c;u times out, originally 25
                         validation_data = test_set,
                         validation_steps = 2000)

# Saving model to disk
classifier.save('model.h5')

## Testing the Model

In [ ]:
#checking if our model works on a single picture
from keras.preprocessing import image
from keras import models
import numpy as np

#loading the trained model
model = models.load_model('model.h5')
#testing on a single picture
test_image=image.load_img('dataset/single_pred/0.jpg',target_size = (64, 64))
#converts it into 3d array
test_image=image.img_to_array(test_image) #converts it into 3d array
test_image=np.expand_dims(test_image,axis=0)

#finding the prediction by inputting the array into the model
prediction=model.predict([test_image])
print(prediction)

## Running the Model Live and Testing Through a Webcam Feed

In [ ]:
import numpy as np
import cv2 
from keras import models

#loading the model
model = models.load_model('model.h5')
#getting the object from the webcam ( default webcam input is at 0)
cap = cv2.VideoCapture(0) 

from PIL import Image
from keras.preprocessing import image

#using haarcascade_frontalface to detect the face
face_cascade = cv2.CascadeClassifier('C:/Users/Tarun/Desktop/comp/projects/face_mask_detector/haarcascades/haarcascade_frontalface_default.xml')

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#main function to detect the face and if a mask is present
def detect_face(img):
    
    #getting a copy of the frame
    face_img = img.copy()
    #detecting the face from the frame
    face_rects = face_cascade.detectMultiScale(face_img)
    
    for (face_x,face_y,w,h) in face_rects:
        #getting the region of interest of the face
        roi = frame[face_y:face_y+h, face_x:face_x+w]
        #resizing to suit our model
        roi = cv2.resize(roi, (64,64))
        #saving as image so as to make changing of sizes and changing to arrays easier for us
        cv2.imwrite('frame.jpg', roi)
        roi=image.load_img('frame.jpg',target_size = (64, 64))
        #converts it into 3d array
        img_array=image.img_to_array(roi) #converts it into 3d array
        img_array=np.expand_dims(img_array,axis=0)
        
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        #predicting by passing array into the model
        predict=model.predict([img_array])
        import winsound     
        import time
        #getting the predicition value from the array
        #if human is not wearing a mask , an alarm starts beeping
        if(predict==[[1.]]):
            prediction="No Mask !!"
            winsound.Beep((1500), 50)
        else:
            prediction="Mask found !"
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.rectangle(img, (face_x,face_y), (face_x+w,face_y+h), (255,255,255), 2)
        cv2.putText(img,prediction,(0,130), font, 1, (200,255,155), 2, cv2.LINE_AA)
        
    return img

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

while True: 
    #getting frames from webcam
    ret, frame = cap.read(0) 
    #passing frames to the emotions detection function
    frame = detect_face(frame)
    #final outputting of the frame 
    cv2.imshow('Mask detection', frame) 
    c = cv2.waitKey(1) 
    if c == 27: 
        break 
        
cap.release() 
cv2.destroyAllWindows()